# 전처리단계

## 중복제거

In [ ]:
import pandas as pd
cheongju = pd.read_csv("./crawled_data/merged_cheongju.csv", encoding="utf-8-sig")
other = pd.read_csv("./crawled_data/merged_other.csv", encoding="utf-8-sig")

In [ ]:
cheongju_duplicated_rows = cheongju[cheongju.duplicated(subset=['title'], keep=False)]
print("중복된 cheongju_title 데이터:", len(cheongju_duplicated_rows)) # 27

other_duplicated_rows = other[other.duplicated(subset=['title'], keep=False)]
print("중복된 other_title 데이터:", len(other_duplicated_rows)) # 2

중복된 cheongju_title 데이터: 27
중복된 other_title 데이터: 2


In [ ]:
cheongju = cheongju.drop_duplicates(subset=['title'], keep='first')
other = other.drop_duplicates(subset=['title'], keep='first')

In [ ]:
cheongju_duplicated_rows = cheongju[cheongju.duplicated(subset=['title'], keep=False)]
print("중복제거 후 중복된 cheongju_title 데이터:", len(cheongju_duplicated_rows))

other_duplicated_rows = other[other.duplicated(subset=['title'], keep=False)]
print("중복제거 후 중복된 other_title 데이터:", len(other_duplicated_rows))

중복제거 후 중복된 cheongju_title 데이터: 0
중복제거 후 중복된 other_title 데이터: 0


In [ ]:
#파일 덮어 씌우기
cheongju.to_csv("./crawled_data/merged_cheongju.csv", index=False, encoding="utf-8-sig")
other.to_csv("./crawled_data/merged_other.csv", index=False, encoding="utf-8-sig")
print("중복 제거 후 파일 저장 완료")

중복 제거 후 파일 저장 완료


## 익명화 및 특수문자 제거

In [28]:
!pip install emoji

  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
Using cached emoji-2.14.1-py3-none-any.whl (590 kB)


In [29]:
import pandas as pd
import emoji

# --- 데이터 안전하게 불러오기 (이전 코드 재사용) ---

def load_csv_robustly(file_path):
    """CSV 파일을 utf-8-sig로 읽되, 실패 시 cp949로 다시 읽는 함수"""
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        print(f"'{file_path}' 파일을 'utf-8-sig'로 성공적으로 불러왔습니다.")
        return df
    except UnicodeDecodeError:
        print(f"'{file_path}' 파일을 'utf-8-sig'로 읽기 실패. 'cp949'로 다시 시도합니다.")
        df = pd.read_csv(file_path, encoding='cp949')
        print(f"'{file_path}' 파일을 'cp949'로 성공적으로 불러왔습니다.")
        return df

print("데이터 로딩 시작...")
cheongju = load_csv_robustly("./crawled_data/merged_cheongju.csv")
other = load_csv_robustly("./crawled_data/merged_other.csv")
print("="*50)

# --- 이모지 제거 작업 ---

# ※※※ 중요 ※※※
# 이모지를 제거할 텍스트가 들어있는 열(column)의 이름을 정확히 지정해주세요.
COLUMN_TO_CLEAN = 'content'

# 텍스트에서 이모지를 제거하는 함수 정의
def remove_emojis(text):
    # emoji.replace_emoji() 함수는 텍스트 안의 모든 이모지를 찾아 replace 인자로 대체합니다.
    # replace='' 로 지정하면 모든 이모지가 삭제됩니다.
    return emoji.replace_emoji(str(text), replace='')

print("이모지 제거 작업 시작...")

# cheongju 데이터프레임에 함수 적용
if COLUMN_TO_CLEAN in cheongju.columns:
    cheongju['content_cleaned'] = cheongju[COLUMN_TO_CLEAN].apply(remove_emojis)
    print("'cheongju' 데이터프레임 이모지 제거 완료.")

# other 데이터프레임에 함수 적용
if COLUMN_TO_CLEAN in other.columns:
    other['content_cleaned'] = other[COLUMN_TO_CLEAN].apply(remove_emojis)
    print("'other' 데이터프레임 이모지 제거 완료.")
print("="*50)


# --- 결과 확인 ---
print("\n--- 청주 데이터프레임 이모지 제거 결과 (상위 5개) ---")
if 'content_cleaned' in cheongju.columns:
    # 원본과 제거 후를 비교하여 출력
    print(cheongju[[COLUMN_TO_CLEAN, 'content_cleaned']].head())

print("\n--- 다른 지역 데이터프레임 이모지 제거 결과 (상위 5개) ---")
if 'content_cleaned' in other.columns:
    print(other[[COLUMN_TO_CLEAN, 'content_cleaned']].head())

데이터 로딩 시작...
'./crawled_data/merged_cheongju.csv' 파일을 'utf-8-sig'로 성공적으로 불러왔습니다.
'./crawled_data/merged_other.csv' 파일을 'utf-8-sig'로 성공적으로 불러왔습니다.
이모지 제거 작업 시작...
'cheongju' 데이터프레임 이모지 제거 완료.
'other' 데이터프레임 이모지 제거 완료.

--- 청주 데이터프레임 이모지 제거 결과 (상위 5개) ---
                                             content  \
0  저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...   
1  ​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...   
2  50m© NAVER Corp.더보기/OpenStreetMap지도 데이터x© NAVE...   
3  청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...   
4  ​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는 편인...   

                                     content_cleaned  
0  저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...  
1  ​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...  
2  50m NAVER Corp.더보기/OpenStreetMap지도 데이터x NAVER ...  
3  청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...  
4  ​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는 편인...  

--- 다른 지역 데이터프레임 이모지 제거 결과 (상위 5개) ---
                                       

## 광고성 글 완전 제거

In [31]:
import pandas as pd
import re

# --- 데이터 안전하게 불러오기 및 기본 전처리 (이전 코드 재사용) ---
def load_csv_robustly(file_path):
    """CSV 파일을 utf-8-sig로 읽되, 실패 시 cp949로 다시 읽는 함수"""
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        return df
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='cp949')
        return df

print("데이터 로딩 시작...")
cheongju = load_csv_robustly("./crawled_data/merged_cheongju.csv")
other = load_csv_robustly("./crawled_data/merged_other.csv")
print("="*50)

COLUMN_TO_CLEAN = 'content' # 실제 사용하는 열 이름으로 변경하세요

if COLUMN_TO_CLEAN in cheongju.columns:
    cheongju[COLUMN_TO_CLEAN] = cheongju[COLUMN_TO_CLEAN].fillna('').astype(str)
if COLUMN_TO_CLEAN in other.columns:
    other[COLUMN_TO_CLEAN] = other[COLUMN_TO_CLEAN].fillna('').astype(str)


# --- 광고성 문구 제거 작업 ---

def remove_ad_text(text):
    """
    텍스트에서 광고성 문구, 전화번호, URL 등을 제거하는 함수
    """
    
    # 1. 제거할 광고성 문구 리스트 (지속적으로 추가/관리 가능)
    # 띄어쓰기가 다르거나 일부만 포함되어도 감지할 수 있도록 핵심 키워드 위주로 작성
    ad_phrases = [
        "소정의 원고료", "소정의 수수료", "제품을 제공받아", "업체로부터", "지원받아 작성",
        "제공받아 솔직하게", "솔직하게 작성한 후기", "체험단으로 선정", "체험단에 선정",
        "파트너스 활동", "일정액의 수수료를", "쿠팡 파트너스", "내돈내산 아님",
        "직접 구매하지 않은", "#협찬", "#광고", "#유료광고", "#광고포함"
    ]

    # 텍스트를 줄 단위로 분리
    lines = text.split('\n')
    clean_lines = []

    for line in lines:
        is_ad_line = False
        # 광고 문구 리스트에 있는 단어가 포함된 줄인지 확인
        for phrase in ad_phrases:
            if phrase in line:
                is_ad_line = True
                break
        
        # 광고 문구가 포함되지 않은 줄만 clean_lines에 추가
        if not is_ad_line:
            clean_lines.append(line)
            
    # 깨끗해진 줄들을 다시 하나의 텍스트로 합침
    text = '\n'.join(clean_lines)

    # 2. 정규표현식을 사용하여 전화번호 제거 (e.g., 010-1234-5678, 043-123-4567)
    phone_pattern = re.compile(r'\d{2,3}-\d{3,4}-\d{4}')
    text = phone_pattern.sub('', text)

    # 3. 정규표현식을 사용하여 URL 제거
    url_pattern = re.compile(r'https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    text = url_pattern.sub('', text)
    
    return text.strip()


# --- 테스트를 위한 예제 텍스트 ---
sample_text = """
정말 맛있는 청주 맛집을 소개합니다.
분위기도 좋고, 음식도 최고였어요.
자세한 문의는 010-1234-5678 로 연락주세요.
더 많은 정보는 http://myblog.com/review 에서 확인 가능합니다.

#청주맛집 #분위기좋은곳

<이 포스팅은 업체로부터 소정의 원고료를 지원받아 작성되었습니다.>
"""

print("--- 광고 제거 함수 테스트 ---")
cleaned_sample = remove_ad_text(sample_text)
print("▶ 원본 텍스트:\n", sample_text)
print("\n▶ 제거 후 텍스트:\n", cleaned_sample)
print("="*50)


# --- 실제 데이터프레임에 함수 적용 ---

print("광고성 문구 제거 작업 시작...")

# cheongju 데이터프레임에 함수 적용
if COLUMN_TO_CLEAN in cheongju.columns:
    # 'content_cleaned' 열이 이미 있다면 그 위에 덮어쓰고, 없다면 새로 생성
    cheongju['content_cleaned'] = cheongju[COLUMN_TO_CLEAN].apply(remove_ad_text)
    print("'cheongju' 데이터프레임 광고 제거 완료.")

# other 데이터프레임에 함수 적용
if COLUMN_TO_CLEAN in other.columns:
    other['content_cleaned'] = other[COLUMN_TO_CLEAN].apply(remove_ad_text)
    print("'other' 데이터프레임 광고 제거 완료.")
print("="*50)

# --- 결과 확인 ---
print("\n--- 청주 데이터프레임 광고 제거 결과 (상위 5개) ---")
if 'content_cleaned' in cheongju.columns:
    print(cheongju[[COLUMN_TO_CLEAN, 'content_cleaned']].head())
print("\n--- 다른 지역 데이터프레임 광고 제거 결과 (상위 5개) ---")
if 'content_cleaned' in other.columns:
    print(other[[COLUMN_TO_CLEAN, 'content_cleaned']].head())

데이터 로딩 시작...
--- 광고 제거 함수 테스트 ---
▶ 원본 텍스트:
 
정말 맛있는 청주 맛집을 소개합니다.
분위기도 좋고, 음식도 최고였어요.
자세한 문의는 010-1234-5678 로 연락주세요.
더 많은 정보는 http://myblog.com/review 에서 확인 가능합니다.

#청주맛집 #분위기좋은곳

<이 포스팅은 업체로부터 소정의 원고료를 지원받아 작성되었습니다.>


▶ 제거 후 텍스트:
 정말 맛있는 청주 맛집을 소개합니다.
분위기도 좋고, 음식도 최고였어요.
자세한 문의는  로 연락주세요.
더 많은 정보는  에서 확인 가능합니다.

#청주맛집 #분위기좋은곳
광고성 문구 제거 작업 시작...
'cheongju' 데이터프레임 광고 제거 완료.
'other' 데이터프레임 광고 제거 완료.

--- 청주 데이터프레임 광고 제거 결과 (상위 5개) ---
                                             content  \
0  저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...   
1  ​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...   
2  50m© NAVER Corp.더보기/OpenStreetMap지도 데이터x© NAVE...   
3  청주 가경동 일식 코스 요리가족 모임하기 좋은 룸식당 어도횟집글, 사진 @네정​​​...   
4  ​​​며칠 전 친구랑 청주 금성식당 다녀왔어요. 원래 고깃집은 워낙 자주 가는 편인...   

                                     content_cleaned  
0  저랑 신랑은더덕구이를 좋아하는대요신랑이 직장동료들이랑점심으로 더덕구이정식을 먹고왔는...  
1  ​청주 가경동 냉면 식당 다녀왔어요출장을 청주로 가면 가경동에서 손님을 만나다 보니...  
2  50m© NAVER Corp.더보기/OpenStreetMap지도 데이터x© NAVE...  
3  청

## 데이터 기간 동일화

In [32]:
import pandas as pd

# --- 데이터 안전하게 불러오기 (이전 코드 재사용) ---
def load_csv_robustly(file_path):
    """CSV 파일을 utf-8-sig로 읽되, 실패 시 cp949로 다시 읽는 함수"""
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        return df
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='cp949')
        return df

print("데이터 로딩 시작...")
cheongju = load_csv_robustly("./crawled_data/merged_cheongju.csv")
other = load_csv_robustly("./crawled_data/merged_other.csv")
print("="*50)

# --- 날짜 기준으로 데이터 필터링 ---

# ※※※ 중요 ※※※
# 날짜가 들어있는 열(column)의 이름을 정확히 지정해주세요.
DATE_COLUMN = 'date'

print(f"필터링 전 'other' 데이터프레임의 정보:")
print(f" - 전체 행 개수: {len(other)}")
print(f" - 가장 오래된 날짜: {other[DATE_COLUMN].min()}")
print("="*50)

# 1. 'date' 열을 날짜(datetime) 형식으로 변환
#    - to_datetime 함수가 YYYYMMDD 형식의 숫자를 날짜로 똑똑하게 변환해 줍니다.
#    - errors='coerce' 옵션은 만약 날짜 형식이 아닌 값이 있을 경우, 해당 값을 NaT(Not a Time)으로 만들어 오류를 방지합니다.
try:
    cheongju[DATE_COLUMN] = pd.to_datetime(cheongju[DATE_COLUMN], format='%Y%m%d', errors='coerce')
    other[DATE_COLUMN] = pd.to_datetime(other[DATE_COLUMN], format='%Y%m%d', errors='coerce')
    print("'date' 열을 날짜(datetime) 형식으로 성공적으로 변환했습니다.")
except Exception as e:
    print(f"'date' 열 변환 중 오류 발생: {e}. 데이터 타입을 확인해주세요.")


# 2. cheongju 데이터프레임에서 기준이 될 최소 날짜(가장 오래된 날짜)를 찾기
min_date_standard = cheongju[DATE_COLUMN].min()
print(f"\n기준 날짜 (cheongju의 최소 날짜): {min_date_standard.strftime('%Y-%m-%d')}")


# 3. other 데이터프레임에서 기준 날짜보다 오래된 데이터를 삭제
#    - other 데이터프레임의 'date'가 기준 날짜보다 크거나 같은 행만 남깁니다.
other_filtered = other[other[DATE_COLUMN] >= min_date_standard].copy()


# 4. 결과 확인
print("\n필터링 후 'other' 데이터프레임의 정보:")
print(f" - 전체 행 개수: {len(other_filtered)}")
print(f" - 가장 오래된 날짜: {other_filtered[DATE_COLUMN].min().strftime('%Y-%m-%d')}")
print(f"-> {len(other) - len(other_filtered)}개의 오래된 데이터가 삭제되었습니다.")
print("="*50)

# 5. 원래의 other 변수에 필터링된 결과 덮어쓰기
other = other_filtered

# 이제 'other' 데이터프레임은 cheongju와 동일한 날짜 기준을 가지게 되었습니다.
print("'other' 데이터프레임이 성공적으로 필터링되었습니다.")

데이터 로딩 시작...
필터링 전 'other' 데이터프레임의 정보:
 - 전체 행 개수: 896
 - 가장 오래된 날짜: 20060824
'date' 열을 날짜(datetime) 형식으로 성공적으로 변환했습니다.

기준 날짜 (cheongju의 최소 날짜): 2013-11-18

필터링 후 'other' 데이터프레임의 정보:
 - 전체 행 개수: 892
 - 가장 오래된 날짜: 2013-11-27
-> 4개의 오래된 데이터가 삭제되었습니다.
'other' 데이터프레임이 성공적으로 필터링되었습니다.
